<h1 style="color:#27391C; font-weight:bold; font-style:italic; font-family:Courier; font-size:30px; "> Preprocessing Scraped Movie Data for Analysis </h1>

In [81]:
import os
import pandas as pd
import numpy as np
import time

from googletrans import Translator
from numerize import numerize as nz

from tqdm.notebook import tqdm, trange
tqdm.pandas()

<h5 style="color:#A64D79;font-style:italic; font-weight:bold">Reading the AZ Movies Dataset</h5>

In [82]:
filename = '../data/raw/azmovies.csv' 
az_movies = pd.read_csv(filename)
az_movies

,Unnamed: 0,Title,Genre,Director,Writer,Language,Production Country,Production Company,User Rating,Release Date,Run Time,IMDB Rating,Metascore,Rotten Tomatoes Score
0,0,Flow,"Drama, Comedy, Horror, Crime",NaN,NaN,English,NaN,NaN,8.0,1996-01-01,1h 20min,4.9,NaN,NaN
1,1,Demon City,"Action, Crime, Fantasy, Thriller",Seiji Tanaka,NaN,日本語,Japan,Netflix,6.8,2025-02-26,1h 46min,NaN,NaN,NaN
2,2,Mickey 17,"Science Fiction, Comedy, Adventure, Fantasy",Bong Joon Ho,NaN,English,"United Kingdom, United States of America",Warner Bros. Pictures,7.1,2025-02-28,2h 17min,7.3,75.0,89.0
3,3,Solo Leveling -ReAwakening-,"Action, Adventure, Fantasy, Animation",Shunsuke Nakashige,NaN,日本語,Japan,Aniplex,6.8,2024-11-26,1h 56min,8.6,NaN,NaN
4,4,Wicked,"Drama, Romance, Fantasy",Jon M. Chu,"Winnie Holzman, Dana Fox",English,United States of America,Universal Pictures,6.9,2024-11-20,2h 42min,8.0,73.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Secret of Moonacre,"Adventure, Fantasy, Romance",Gábor Csupó,"Lucy Shuttleworth, Graham Alborough",English,"Australia, France, Hungary, United States of A...",Forgan-Smith Entertainment,6.5,2009-02-06,1h 43min,6.0,23.0,NaN
996,996,Girl Haunts Boy,"Fantasy, Romance, Comedy",Emily Ting,NaN,English,"South Korea, United States of America",Fifth Season,6.8,2024-10-09,1h 40min,6.3,92.0,NaN
997,997,Sumuru,"Science Fiction, Fantasy, Adventure, Action",Darrell James Roodt,"Peter Jobin, Harry Alan Towers, Torsten Dewi",English,"Canada, Germany, South Africa, United Kingdom",Tandem Communications,5.3,2003-04-30,1h 30min,3.6,NaN,NaN
998,998,The Man from the Future,"Comedy, Fantasy, Science Fiction, Romance",Cláudio Torres,Cláudio Torres,Português,Brazil,Globo Filmes,7.2,2011-09-02,1h 46min,7.0,NaN,NaN


In [83]:
az_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1000 non-null   int64  
 1   Title                  970 non-null    object 
 2   Genre                  968 non-null    object 
 3   Director               776 non-null    object 
 4   Writer                 679 non-null    object 
 5   Language               963 non-null    object 
 6   Production Country     935 non-null    object 
 7   Production Company     956 non-null    object 
 8   User Rating            944 non-null    float64
 9   Release Date           970 non-null    object 
 10  Run Time               970 non-null    object 
 11  IMDB Rating            924 non-null    float64
 12  Metascore              716 non-null    float64
 13  Rotten Tomatoes Score  591 non-null    float64
dtypes: float64(4), int64(1), object(9)
memory usage: 109.5+ K

In [84]:
len(az_movies)

1000

In [85]:
print(az_movies.columns)


Index(['Unnamed: 0', 'Title', 'Genre', 'Director', 'Writer', 'Language',
       'Production Country', 'Production Company', 'User Rating',
       'Release Date', 'Run Time', 'IMDB Rating', 'Metascore',
       'Rotten Tomatoes Score'],
      dtype='object')


<h5 style="color:#A64D79;font-style:italic; font-weight:bold">Removing Duplicates and Renaming Columns in Movie Data</h5>

In [86]:
az_movies = az_movies.drop_duplicates()




az_movies = az_movies.rename(columns={"Unnamed: 0": "title_id"})

az_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title_id               1000 non-null   int64  
 1   Title                  970 non-null    object 
 2   Genre                  968 non-null    object 
 3   Director               776 non-null    object 
 4   Writer                 679 non-null    object 
 5   Language               963 non-null    object 
 6   Production Country     935 non-null    object 
 7   Production Company     956 non-null    object 
 8   User Rating            944 non-null    float64
 9   Release Date           970 non-null    object 
 10  Run Time               970 non-null    object 
 11  IMDB Rating            924 non-null    float64
 12  Metascore              716 non-null    float64
 13  Rotten Tomatoes Score  591 non-null    float64
dtypes: float64(4), int64(1), object(9)
memory usage: 109.5+ K

In [87]:
az_movies = (
    az_movies.rename(columns={
        'Production Country': 'pro_country',
        'Production Company': 'pro_company',
        'User Rating': 'user_rating',
        'Release Date': 'release_date',
        'Run Time': 'run_time',
        'IMDB Rating': 'imdb_rating',
        'Rotten Tomatoes Score':'rt_score'
    })
)

In [88]:
az_movies

,title_id,Title,Genre,Director,Writer,Language,pro_country,pro_company,user_rating,release_date,run_time,imdb_rating,Metascore,rt_score
0,0,Flow,"Drama, Comedy, Horror, Crime",NaN,NaN,English,NaN,NaN,8.0,1996-01-01,1h 20min,4.9,NaN,NaN
1,1,Demon City,"Action, Crime, Fantasy, Thriller",Seiji Tanaka,NaN,日本語,Japan,Netflix,6.8,2025-02-26,1h 46min,NaN,NaN,NaN
2,2,Mickey 17,"Science Fiction, Comedy, Adventure, Fantasy",Bong Joon Ho,NaN,English,"United Kingdom, United States of America",Warner Bros. Pictures,7.1,2025-02-28,2h 17min,7.3,75.0,89.0
3,3,Solo Leveling -ReAwakening-,"Action, Adventure, Fantasy, Animation",Shunsuke Nakashige,NaN,日本語,Japan,Aniplex,6.8,2024-11-26,1h 56min,8.6,NaN,NaN
4,4,Wicked,"Drama, Romance, Fantasy",Jon M. Chu,"Winnie Holzman, Dana Fox",English,United States of America,Universal Pictures,6.9,2024-11-20,2h 42min,8.0,73.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Secret of Moonacre,"Adventure, Fantasy, Romance",Gábor Csupó,"Lucy Shuttleworth, Graham Alborough",English,"Australia, France, Hungary, United States of A...",Forgan-Smith Entertainment,6.5,2009-02-06,1h 43min,6.0,23.0,NaN
996,996,Girl Haunts Boy,"Fantasy, Romance, Comedy",Emily Ting,NaN,English,"South Korea, United States of America",Fifth Season,6.8,2024-10-09,1h 40min,6.3,92.0,NaN
997,997,Sumuru,"Science Fiction, Fantasy, Adventure, Action",Darrell James Roodt,"Peter Jobin, Harry Alan Towers, Torsten Dewi",English,"Canada, Germany, South Africa, United Kingdom",Tandem Communications,5.3,2003-04-30,1h 30min,3.6,NaN,NaN
998,998,The Man from the Future,"Comedy, Fantasy, Science Fiction, Romance",Cláudio Torres,Cláudio Torres,Português,Brazil,Globo Filmes,7.2,2011-09-02,1h 46min,7.0,NaN,NaN


<h5 style="color:#A64D79;font-style:italic; font-weight:bold">Cleaning Whitespace Characters in the 'Production Company' Column</h5>

In [89]:

az_movies.loc[az_movies['pro_company'].str.contains(r'\n|\t', na=False), 'pro_company'] = \
    az_movies['pro_company'].str.replace(r'\n|\t', '', regex=True)

az_movies


,title_id,Title,Genre,Director,Writer,Language,pro_country,pro_company,user_rating,release_date,run_time,imdb_rating,Metascore,rt_score
0,0,Flow,"Drama, Comedy, Horror, Crime",NaN,NaN,English,NaN,NaN,8.0,1996-01-01,1h 20min,4.9,NaN,NaN
1,1,Demon City,"Action, Crime, Fantasy, Thriller",Seiji Tanaka,NaN,日本語,Japan,Netflix,6.8,2025-02-26,1h 46min,NaN,NaN,NaN
2,2,Mickey 17,"Science Fiction, Comedy, Adventure, Fantasy",Bong Joon Ho,NaN,English,"United Kingdom, United States of America",Warner Bros. Pictures,7.1,2025-02-28,2h 17min,7.3,75.0,89.0
3,3,Solo Leveling -ReAwakening-,"Action, Adventure, Fantasy, Animation",Shunsuke Nakashige,NaN,日本語,Japan,Aniplex,6.8,2024-11-26,1h 56min,8.6,NaN,NaN
4,4,Wicked,"Drama, Romance, Fantasy",Jon M. Chu,"Winnie Holzman, Dana Fox",English,United States of America,Universal Pictures,6.9,2024-11-20,2h 42min,8.0,73.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Secret of Moonacre,"Adventure, Fantasy, Romance",Gábor Csupó,"Lucy Shuttleworth, Graham Alborough",English,"Australia, France, Hungary, United States of A...",Forgan-Smith Entertainment,6.5,2009-02-06,1h 43min,6.0,23.0,NaN
996,996,Girl Haunts Boy,"Fantasy, Romance, Comedy",Emily Ting,NaN,English,"South Korea, United States of America",Fifth Season,6.8,2024-10-09,1h 40min,6.3,92.0,NaN
997,997,Sumuru,"Science Fiction, Fantasy, Adventure, Action",Darrell James Roodt,"Peter Jobin, Harry Alan Towers, Torsten Dewi",English,"Canada, Germany, South Africa, United Kingdom",Tandem Communications,5.3,2003-04-30,1h 30min,3.6,NaN,NaN
998,998,The Man from the Future,"Comedy, Fantasy, Science Fiction, Romance",Cláudio Torres,Cláudio Torres,Português,Brazil,Globo Filmes,7.2,2011-09-02,1h 46min,7.0,NaN,NaN


<h5 style="color:#A64D79;font-style:italic; font-weight:bold">Using Google Translator to Convert Language Data to English</h5>

In [90]:
print(az_movies['Language'].unique())


['English' '日本語' 'Français' 'Dansk' 'Español' '普通话' 'Deutsch' '广州话 / 廣州話'
 nan '한국어/조선말' 'తెలుగు' 'Nederlands' 'svenska' 'No Language' 'தமிழ்'
 'العربية' 'Italiano' 'ภาษาไทย' 'Pусский' 'Íslenska' 'Український'
 'Bahasa indonesia' 'ελληνικά' 'हिन्दी' 'Český' 'Português' 'Latin'
 'Polski' 'Magyar']


In [91]:
translator = Translator()


unique_languages = az_movies['Language'].dropna().unique()


translation_dict = {}


for lang in unique_languages:
    try:
        translated_text = translator.translate(lang, dest='en').text
        translation_dict[lang] = translated_text
        time.sleep(0.5) 
    except Exception as e:
        print(f"Error translating '{lang}': {e}")
        translation_dict[lang] = lang  


az_movies['Language'] = az_movies['Language'].map(translation_dict)


print(az_movies['Language'].unique())


Error translating 'Deutsch': [WinError 10054] An existing connection was forcibly closed by the remote host
['English' 'Japanese' 'French' 'Danish' 'Español' 'mandarin' 'Deutsch'
 'Guangzhou dialect / Guangzhou dialect' nan 'Korean/Joseon' 'Telugu'
 'Dutch' 'Swedish' 'No Language' 'Tamil' 'Arabic' 'Italian'
 'Thai language' 'Pusian' 'Icelandic' 'Ukrainian' 'Indonesian' 'Greek'
 'Hindi' 'Czech' 'Portuguese' 'Latin' 'Polish' 'Hungarian']


In [92]:
az_movies

,title_id,Title,Genre,Director,Writer,Language,pro_country,pro_company,user_rating,release_date,run_time,imdb_rating,Metascore,rt_score
0,0,Flow,"Drama, Comedy, Horror, Crime",NaN,NaN,English,NaN,NaN,8.0,1996-01-01,1h 20min,4.9,NaN,NaN
1,1,Demon City,"Action, Crime, Fantasy, Thriller",Seiji Tanaka,NaN,Japanese,Japan,Netflix,6.8,2025-02-26,1h 46min,NaN,NaN,NaN
2,2,Mickey 17,"Science Fiction, Comedy, Adventure, Fantasy",Bong Joon Ho,NaN,English,"United Kingdom, United States of America",Warner Bros. Pictures,7.1,2025-02-28,2h 17min,7.3,75.0,89.0
3,3,Solo Leveling -ReAwakening-,"Action, Adventure, Fantasy, Animation",Shunsuke Nakashige,NaN,Japanese,Japan,Aniplex,6.8,2024-11-26,1h 56min,8.6,NaN,NaN
4,4,Wicked,"Drama, Romance, Fantasy",Jon M. Chu,"Winnie Holzman, Dana Fox",English,United States of America,Universal Pictures,6.9,2024-11-20,2h 42min,8.0,73.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Secret of Moonacre,"Adventure, Fantasy, Romance",Gábor Csupó,"Lucy Shuttleworth, Graham Alborough",English,"Australia, France, Hungary, United States of A...",Forgan-Smith Entertainment,6.5,2009-02-06,1h 43min,6.0,23.0,NaN
996,996,Girl Haunts Boy,"Fantasy, Romance, Comedy",Emily Ting,NaN,English,"South Korea, United States of America",Fifth Season,6.8,2024-10-09,1h 40min,6.3,92.0,NaN
997,997,Sumuru,"Science Fiction, Fantasy, Adventure, Action",Darrell James Roodt,"Peter Jobin, Harry Alan Towers, Torsten Dewi",English,"Canada, Germany, South Africa, United Kingdom",Tandem Communications,5.3,2003-04-30,1h 30min,3.6,NaN,NaN
998,998,The Man from the Future,"Comedy, Fantasy, Science Fiction, Romance",Cláudio Torres,Cláudio Torres,Portuguese,Brazil,Globo Filmes,7.2,2011-09-02,1h 46min,7.0,NaN,NaN


In [93]:
az_movies['Language'].unique()

array(['English', 'Japanese', 'French', 'Danish', 'Español', 'mandarin',
       'Deutsch', 'Guangzhou dialect / Guangzhou dialect', nan,
       'Korean/Joseon', 'Telugu', 'Dutch', 'Swedish', 'No Language',
       'Tamil', 'Arabic', 'Italian', 'Thai language', 'Pusian',
       'Icelandic', 'Ukrainian', 'Indonesian', 'Greek', 'Hindi', 'Czech',
       'Portuguese', 'Latin', 'Polish', 'Hungarian'], dtype=object)

<h5 style="color:#A64D79;font-style:italic; font-weight:bold">Using String Manipulation and strip() to Extract and Clean Primary Language</h5>

In [94]:
az_movies['Language'] = az_movies['Language'].astype(str).str.split('/').str[0].str.strip()
az_movies

,title_id,Title,Genre,Director,Writer,Language,pro_country,pro_company,user_rating,release_date,run_time,imdb_rating,Metascore,rt_score
0,0,Flow,"Drama, Comedy, Horror, Crime",NaN,NaN,English,NaN,NaN,8.0,1996-01-01,1h 20min,4.9,NaN,NaN
1,1,Demon City,"Action, Crime, Fantasy, Thriller",Seiji Tanaka,NaN,Japanese,Japan,Netflix,6.8,2025-02-26,1h 46min,NaN,NaN,NaN
2,2,Mickey 17,"Science Fiction, Comedy, Adventure, Fantasy",Bong Joon Ho,NaN,English,"United Kingdom, United States of America",Warner Bros. Pictures,7.1,2025-02-28,2h 17min,7.3,75.0,89.0
3,3,Solo Leveling -ReAwakening-,"Action, Adventure, Fantasy, Animation",Shunsuke Nakashige,NaN,Japanese,Japan,Aniplex,6.8,2024-11-26,1h 56min,8.6,NaN,NaN
4,4,Wicked,"Drama, Romance, Fantasy",Jon M. Chu,"Winnie Holzman, Dana Fox",English,United States of America,Universal Pictures,6.9,2024-11-20,2h 42min,8.0,73.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Secret of Moonacre,"Adventure, Fantasy, Romance",Gábor Csupó,"Lucy Shuttleworth, Graham Alborough",English,"Australia, France, Hungary, United States of A...",Forgan-Smith Entertainment,6.5,2009-02-06,1h 43min,6.0,23.0,NaN
996,996,Girl Haunts Boy,"Fantasy, Romance, Comedy",Emily Ting,NaN,English,"South Korea, United States of America",Fifth Season,6.8,2024-10-09,1h 40min,6.3,92.0,NaN
997,997,Sumuru,"Science Fiction, Fantasy, Adventure, Action",Darrell James Roodt,"Peter Jobin, Harry Alan Towers, Torsten Dewi",English,"Canada, Germany, South Africa, United Kingdom",Tandem Communications,5.3,2003-04-30,1h 30min,3.6,NaN,NaN
998,998,The Man from the Future,"Comedy, Fantasy, Science Fiction, Romance",Cláudio Torres,Cláudio Torres,Portuguese,Brazil,Globo Filmes,7.2,2011-09-02,1h 46min,7.0,NaN,NaN


<h5 style="color:#A64D79;font-style:italic; font-weight:bold;"> Using .unique() to Display Unique Production Companies</h5>

In [95]:
az_movies["pro_company"].unique()


array([nan, ' Netflix', ' Warner Bros. Pictures', ' Aniplex',
       ' Universal Pictures', ' Seven Bucks Productions',
       ' New Line Cinema', ' Walt Disney Pictures', ' Zagtoon',
       ' Pacific Data Images', ' Animaker', ' Studio Mir',
       ' DreamWorks Animation', ' Focus Features',
       ' Chengdu Coco Cartoon', ' Dune Entertainment',
       ' Beijing Enlight Pictures', ' iQIYI', ' Laika', ' Madhouse',
       ' Paramount Pictures', ' Pixar', ' Davis Films', ' Skydance Media',
       ' Summit Entertainment', ' Constantin Film', ' ufotable',
       ' Walt Disney Animation Studios', ' Marvel Studios', ' KADOKAWA',
       ' Nickelodeon Movies', ' MAPPA', ' Studio Ghibli',
       ' Castle Rock Entertainment', ' Metro-Goldwyn-Mayer',
       ' Paranoid Android Films', ' Jerry Bruckheimer Films',
       ' La Persiana Films', ' Suzanne Todd Productions',
       ' 上海摩天之眼文化传媒有限公司', ' 2 Minutes', ' Smart Entertainment',
       ' PCMA Productions', ' Movistar+', ' Marvel Enterprises',
 

<h5 style="color:#A64D79;font-style:italic; font-weight:bold;">Using Google Translator to Standardize Production Company Names</h5>

In [96]:
translator = Translator()

unique_companies = az_movies['pro_company'].dropna().unique()

def translate_to_english(text):
    try:
        return translator.translate(text, dest='en').text 
    except:
        return text  


translation_map = {company: translate_to_english(company) for company in unique_companies}


az_movies['pro_company'] = az_movies['pro_company'].map(translation_map)

print(az_movies['pro_company'].unique())


[nan 'Netflix' 'Warner Bros. Pictures' 'Aniplex' 'Universal Pictures'
 'Seven Bucks Productions' 'New Line Cinema' 'Walt Disney Pictures'
 'Zestone' 'Pacific Data Images' 'Animaker' 'Studio me'
 'DreamWorks Animation' 'Focus Features' 'Chengdu Coco Cartoon'
 'Dune Entertainment' 'Beijing Enlight Pictures' 'IQIYI' 'Time' 'Madhouse'
 'Paramount Pictures' 'Pixar' 'Davis Films' 'Skydance media'
 'Summit Entertainment' 'Constantin Film' 'ufotable'
 'Walt Disney Animation Studios' 'Marvel Studios' 'Kadokawa'
 'Nickelodeon Movies' 'MAP' 'Studio Ghibli' 'Castle Rock Entertainment'
 'Metro-Goldwyn-Mayer' 'Paranoid Android Films' 'Jerry Bruckheimer Films'
 'The Blind Films' 'Suzanne Todd Productions'
 'Shanghai Ferris Eye Culture Media Co., Ltd.' '2 Minutes'
 'Smart Entertainment' 'PCMA Productions' 'Movistar+' 'Marvel Enterprises'
 '1492 Pictures' 'Cinemalaya Foundation' 'Caper Film'
 'Walt Disney Productions' 'The Awakening Production' 'Entertainment One'
 'Michael de Luca Productions' 'Dark H

<h5 style="color:#A64D79;font-style:italic; font-weight:bold;">Using Regex to Replace Empty Strings and NaN Values with None</h5>

In [97]:
az_movies = az_movies.replace(r'^\s*$', None, regex=True).replace({np.nan: None})



In [98]:
az_movies

,title_id,Title,Genre,Director,Writer,Language,pro_country,pro_company,user_rating,release_date,run_time,imdb_rating,Metascore,rt_score
0,0,Flow,"Drama, Comedy, Horror, Crime",None,None,English,None,None,8.0,1996-01-01,1h 20min,4.9,None,None
1,1,Demon City,"Action, Crime, Fantasy, Thriller",Seiji Tanaka,None,Japanese,Japan,Netflix,6.8,2025-02-26,1h 46min,None,None,None
2,2,Mickey 17,"Science Fiction, Comedy, Adventure, Fantasy",Bong Joon Ho,None,English,"United Kingdom, United States of America",Warner Bros. Pictures,7.1,2025-02-28,2h 17min,7.3,75.0,89.0
3,3,Solo Leveling -ReAwakening-,"Action, Adventure, Fantasy, Animation",Shunsuke Nakashige,None,Japanese,Japan,Aniplex,6.8,2024-11-26,1h 56min,8.6,None,None
4,4,Wicked,"Drama, Romance, Fantasy",Jon M. Chu,"Winnie Holzman, Dana Fox",English,United States of America,Universal Pictures,6.9,2024-11-20,2h 42min,8.0,73.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Secret of Moonacre,"Adventure, Fantasy, Romance",Gábor Csupó,"Lucy Shuttleworth, Graham Alborough",English,"Australia, France, Hungary, United States of A...",Forgan-Smith Entertainment,6.5,2009-02-06,1h 43min,6.0,23.0,None
996,996,Girl Haunts Boy,"Fantasy, Romance, Comedy",Emily Ting,None,English,"South Korea, United States of America",Fifth Season,6.8,2024-10-09,1h 40min,6.3,92.0,None
997,997,Sumuru,"Science Fiction, Fantasy, Adventure, Action",Darrell James Roodt,"Peter Jobin, Harry Alan Towers, Torsten Dewi",English,"Canada, Germany, South Africa, United Kingdom",Tandem Communications,5.3,2003-04-30,1h 30min,3.6,None,None
998,998,The Man from the Future,"Comedy, Fantasy, Science Fiction, Romance",Cláudio Torres,Cláudio Torres,Portuguese,Brazil,Globo Movies,7.2,2011-09-02,1h 46min,7.0,None,None


<h5 style="color:#A64D79;font-style:italic; font-weight:bold;">Removing Rows with Missing Titles</h5>

In [99]:
az_movies = az_movies.dropna(subset=['Title'])


In [100]:
az_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 970 entries, 0 to 999
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title_id      970 non-null    int64 
 1   Title         970 non-null    object
 2   Genre         968 non-null    object
 3   Director      776 non-null    object
 4   Writer        679 non-null    object
 5   Language      970 non-null    object
 6   pro_country   935 non-null    object
 7   pro_company   956 non-null    object
 8   user_rating   944 non-null    object
 9   release_date  970 non-null    object
 10  run_time      970 non-null    object
 11  imdb_rating   924 non-null    object
 12  Metascore     716 non-null    object
 13  rt_score      591 non-null    object
dtypes: int64(1), object(13)
memory usage: 113.7+ KB


<h5 style="color:#A64D79;font-style:italic; font-weight:bold;">Replacing Missing Ratings with Zero</h5>

In [101]:
columns_to_fill = ['user_rating', 'imdb_rating', 'Metascore', 'rt_score']
az_movies.loc[:, columns_to_fill] = az_movies.loc[:, columns_to_fill].fillna(0)
az_movies


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_376\3591729230.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  az_movies.loc[:, columns_to_fill] = az_movies.loc[:, columns_to_fill].fillna(0)


,title_id,Title,Genre,Director,Writer,Language,pro_country,pro_company,user_rating,release_date,run_time,imdb_rating,Metascore,rt_score
0,0,Flow,"Drama, Comedy, Horror, Crime",None,None,English,None,None,8.0,1996-01-01,1h 20min,4.9,0.0,0.0
1,1,Demon City,"Action, Crime, Fantasy, Thriller",Seiji Tanaka,None,Japanese,Japan,Netflix,6.8,2025-02-26,1h 46min,0.0,0.0,0.0
2,2,Mickey 17,"Science Fiction, Comedy, Adventure, Fantasy",Bong Joon Ho,None,English,"United Kingdom, United States of America",Warner Bros. Pictures,7.1,2025-02-28,2h 17min,7.3,75.0,89.0
3,3,Solo Leveling -ReAwakening-,"Action, Adventure, Fantasy, Animation",Shunsuke Nakashige,None,Japanese,Japan,Aniplex,6.8,2024-11-26,1h 56min,8.6,0.0,0.0
4,4,Wicked,"Drama, Romance, Fantasy",Jon M. Chu,"Winnie Holzman, Dana Fox",English,United States of America,Universal Pictures,6.9,2024-11-20,2h 42min,8.0,73.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Secret of Moonacre,"Adventure, Fantasy, Romance",Gábor Csupó,"Lucy Shuttleworth, Graham Alborough",English,"Australia, France, Hungary, United States of A...",Forgan-Smith Entertainment,6.5,2009-02-06,1h 43min,6.0,23.0,0.0
996,996,Girl Haunts Boy,"Fantasy, Romance, Comedy",Emily Ting,None,English,"South Korea, United States of America",Fifth Season,6.8,2024-10-09,1h 40min,6.3,92.0,0.0
997,997,Sumuru,"Science Fiction, Fantasy, Adventure, Action",Darrell James Roodt,"Peter Jobin, Harry Alan Towers, Torsten Dewi",English,"Canada, Germany, South Africa, United Kingdom",Tandem Communications,5.3,2003-04-30,1h 30min,3.6,0.0,0.0
998,998,The Man from the Future,"Comedy, Fantasy, Science Fiction, Romance",Cláudio Torres,Cláudio Torres,Portuguese,Brazil,Globo Movies,7.2,2011-09-02,1h 46min,7.0,0.0,0.0


<h5 style="color:#A64D79;font-style:italic; font-weight:bold;">Using .astype(float) to Convert Rating Columns to Numeric Data</h5>

In [102]:
cols_to_convert = ['user_rating', 'imdb_rating', 'Metascore', 'rt_score']

for col in cols_to_convert:
    az_movies.loc[:, col] = az_movies.loc[:, col].astype(float, errors='ignore')


In [103]:
az_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 970 entries, 0 to 999
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title_id      970 non-null    int64 
 1   Title         970 non-null    object
 2   Genre         968 non-null    object
 3   Director      776 non-null    object
 4   Writer        679 non-null    object
 5   Language      970 non-null    object
 6   pro_country   935 non-null    object
 7   pro_company   956 non-null    object
 8   user_rating   970 non-null    object
 9   release_date  970 non-null    object
 10  run_time      970 non-null    object
 11  imdb_rating   970 non-null    object
 12  Metascore     970 non-null    object
 13  rt_score      970 non-null    object
dtypes: int64(1), object(13)
memory usage: 113.7+ KB


In [104]:
az_movies['pro_country'].unique()

array([None, 'Japan', 'United Kingdom, United States of America',
       'United States of America',
       'Japan, New Zealand, United States of America', 'France',
       'Denmark, Finland, Germany, Ireland',
       'Poland, South Korea, United States of America',
       'DreamWorks Animation', 'New Line Cinema', 'Warner Bros. Pictures',
       'China', 'New Zealand, United States of America',
       'Universal Pictures', 'United States of America, United Kingdom',
       'iQIYI', 'Laika', 'Walt Disney Pictures', 'Paramount Pictures',
       'Davis Films', 'Spain, United States of America',
       'Germany, Switzerland', 'Czech Republic, Germany', 'KADOKAWA',
       'Studio Ghibli', 'Castle Rock Entertainment',
       'Suzanne Todd Productions',
       'Canada, France, South Korea, United Kingdom', '上海摩天之眼文化传媒有限公司',
       'ufotable', 'Canada, France', 'Movistar+',
       'Switzerland, United States of America', 'Cinemalaya Foundation',
       'Caper Film', 'China, France, Canada', '

In [105]:
az_movies

,title_id,Title,Genre,Director,Writer,Language,pro_country,pro_company,user_rating,release_date,run_time,imdb_rating,Metascore,rt_score
0,0,Flow,"Drama, Comedy, Horror, Crime",None,None,English,None,None,8.0,1996-01-01,1h 20min,4.9,0.0,0.0
1,1,Demon City,"Action, Crime, Fantasy, Thriller",Seiji Tanaka,None,Japanese,Japan,Netflix,6.8,2025-02-26,1h 46min,0.0,0.0,0.0
2,2,Mickey 17,"Science Fiction, Comedy, Adventure, Fantasy",Bong Joon Ho,None,English,"United Kingdom, United States of America",Warner Bros. Pictures,7.1,2025-02-28,2h 17min,7.3,75.0,89.0
3,3,Solo Leveling -ReAwakening-,"Action, Adventure, Fantasy, Animation",Shunsuke Nakashige,None,Japanese,Japan,Aniplex,6.8,2024-11-26,1h 56min,8.6,0.0,0.0
4,4,Wicked,"Drama, Romance, Fantasy",Jon M. Chu,"Winnie Holzman, Dana Fox",English,United States of America,Universal Pictures,6.9,2024-11-20,2h 42min,8.0,73.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Secret of Moonacre,"Adventure, Fantasy, Romance",Gábor Csupó,"Lucy Shuttleworth, Graham Alborough",English,"Australia, France, Hungary, United States of A...",Forgan-Smith Entertainment,6.5,2009-02-06,1h 43min,6.0,23.0,0.0
996,996,Girl Haunts Boy,"Fantasy, Romance, Comedy",Emily Ting,None,English,"South Korea, United States of America",Fifth Season,6.8,2024-10-09,1h 40min,6.3,92.0,0.0
997,997,Sumuru,"Science Fiction, Fantasy, Adventure, Action",Darrell James Roodt,"Peter Jobin, Harry Alan Towers, Torsten Dewi",English,"Canada, Germany, South Africa, United Kingdom",Tandem Communications,5.3,2003-04-30,1h 30min,3.6,0.0,0.0
998,998,The Man from the Future,"Comedy, Fantasy, Science Fiction, Romance",Cláudio Torres,Cláudio Torres,Portuguese,Brazil,Globo Movies,7.2,2011-09-02,1h 46min,7.0,0.0,0.0


<h5 style="color:#A64D79;font-style:italic; font-weight:bold;">Filtering and Cleaning Production Countries</h5>

In [106]:
valid_countries = [
    "United States of America", "India", "France", "Germany", "Japan", "Mexico", 
    "Spain", "Indonesia", "China", "Belgium", "United Kingdom", "Italy", "Thailand", 
    "Switzerland", "Cyprus", "Denmark", "Finland", "Ireland", "South Korea", 
    "New Zealand", "Portugal", "Hong Kong", "Poland","Sweden","Peru","Norway","Ukraine"
]  # Add more

# Ensure 'pro_country' exists in az_movies df
if 'pro_country' in az_movies.columns:
    # Convert 'pro_country' to string type
    az_movies.loc[:, 'pro_country'] = az_movies['pro_country'].astype(str)

    # Replace values that are not in valid_countries with None
    az_movies.loc[~az_movies['pro_country'].isin(valid_countries), 'pro_country'] = None

    print(az_movies['pro_country'].unique())
else:
    print("Column 'pro_country' not found in az_movies.")


[None 'Japan' 'United States of America' 'France' 'China' 'India'
 'United Kingdom' 'Thailand' 'Indonesia' 'South Korea' 'Poland' 'Germany'
 'Hong Kong' 'Spain' 'Ukraine' 'Norway' 'Mexico' 'Sweden' 'Italy' 'Peru']


In [107]:
az_movies["pro_country"].unique()

array([None, 'Japan', 'United States of America', 'France', 'China',
       'India', 'United Kingdom', 'Thailand', 'Indonesia', 'South Korea',
       'Poland', 'Germany', 'Hong Kong', 'Spain', 'Ukraine', 'Norway',
       'Mexico', 'Sweden', 'Italy', 'Peru'], dtype=object)

<h5 style="color:#A64D79;font-style:italic; font-weight:bold;">Handling Matching Production Country and Company</h5>

In [108]:

az_movies['pro_country'] = az_movies['pro_country'].mask(az_movies['pro_country'] == az_movies['pro_company'], np.nan)



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_376\1827384664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  az_movies['pro_country'] = az_movies['pro_country'].mask(az_movies['pro_country'] == az_movies['pro_company'], np.nan)


In [109]:
az_movies

,title_id,Title,Genre,Director,Writer,Language,pro_country,pro_company,user_rating,release_date,run_time,imdb_rating,Metascore,rt_score
0,0,Flow,"Drama, Comedy, Horror, Crime",None,None,English,None,None,8.0,1996-01-01,1h 20min,4.9,0.0,0.0
1,1,Demon City,"Action, Crime, Fantasy, Thriller",Seiji Tanaka,None,Japanese,Japan,Netflix,6.8,2025-02-26,1h 46min,0.0,0.0,0.0
2,2,Mickey 17,"Science Fiction, Comedy, Adventure, Fantasy",Bong Joon Ho,None,English,None,Warner Bros. Pictures,7.1,2025-02-28,2h 17min,7.3,75.0,89.0
3,3,Solo Leveling -ReAwakening-,"Action, Adventure, Fantasy, Animation",Shunsuke Nakashige,None,Japanese,Japan,Aniplex,6.8,2024-11-26,1h 56min,8.6,0.0,0.0
4,4,Wicked,"Drama, Romance, Fantasy",Jon M. Chu,"Winnie Holzman, Dana Fox",English,United States of America,Universal Pictures,6.9,2024-11-20,2h 42min,8.0,73.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Secret of Moonacre,"Adventure, Fantasy, Romance",Gábor Csupó,"Lucy Shuttleworth, Graham Alborough",English,None,Forgan-Smith Entertainment,6.5,2009-02-06,1h 43min,6.0,23.0,0.0
996,996,Girl Haunts Boy,"Fantasy, Romance, Comedy",Emily Ting,None,English,None,Fifth Season,6.8,2024-10-09,1h 40min,6.3,92.0,0.0
997,997,Sumuru,"Science Fiction, Fantasy, Adventure, Action",Darrell James Roodt,"Peter Jobin, Harry Alan Towers, Torsten Dewi",English,None,Tandem Communications,5.3,2003-04-30,1h 30min,3.6,0.0,0.0
998,998,The Man from the Future,"Comedy, Fantasy, Science Fiction, Romance",Cláudio Torres,Cláudio Torres,Portuguese,None,Globo Movies,7.2,2011-09-02,1h 46min,7.0,0.0,0.0


<h5 style="color:#A64D79;font-style:italic; font-weight:bold;">Using Explode Function to Normalize Multi-Value Columns</h5>

In [110]:
import pandas as pd
import ast

def explode_column(az_movies, column_name, new_column_name):
    df_expanded = az_movies[['title_id', column_name]].copy()  
        
    def safe_convert_list(value):
        if isinstance(value, str):  
            try:
                return ast.literal_eval(value) if value.startswith('[') else value.split(', ')
            except (SyntaxError, ValueError):
                return value.split(', ')  
        elif isinstance(value, list):
            return value  
        else:
            return [value]  

    df_expanded[column_name] = df_expanded[column_name].apply(safe_convert_list)
    df_expanded = df_expanded.explode(column_name).dropna().reset_index(drop=True)
    df_expanded.rename(columns={column_name: new_column_name}, inplace=True)
    return df_expanded

writers_df = explode_column(az_movies, 'Writer', 'writer_name')
genres_df = explode_column(az_movies, 'Genre', 'genre')
production_companies_df = explode_column(az_movies, 'pro_company', 'company_name')
languages_df = explode_column(az_movies, 'Language', 'languages')
country_df = explode_column(az_movies, 'pro_country', 'country')
director_df = explode_column(az_movies,'Director','director_name')

az_movies = az_movies.drop(columns=['Writer', 'Genre', 'pro_company', 'Language', 'pro_country','Director'])


print("Genres DataFrame:\n", genres_df)
print("Writers DataFrame:\n", writers_df)
print("Languages DataFrame:\n", languages_df)
print("Production Companies DataFrame:\n", production_companies_df)
print("Origin Countries DataFrame:\n", country_df)
print("director DataFrame:\n",director_df)


Genres DataFrame:
       title_id            genre
0            0            Drama
1            0           Comedy
2            0           Horror
3            0            Crime
4            1           Action
...        ...              ...
3501       999          Fantasy
3502       999            Drama
3503       999           Comedy
3504       999  Science Fiction
3505       999          Romance

[3506 rows x 2 columns]
Writers DataFrame:
       title_id        writer_name
0            4     Winnie Holzman
1            4           Dana Fox
2            5       Chris Morgan
3            6     Phoebe Gittins
4            6  Arty Papageorgiou
...        ...                ...
1193       995   Graham Alborough
1194       997        Peter Jobin
1195       997  Harry Alan Towers
1196       997       Torsten Dewi
1197       998     Cláudio Torres

[1198 rows x 2 columns]
Languages DataFrame:
      title_id   languages
0           0     English
1           1    Japanese
2           2     E

In [111]:
def add_unique_id(df, name_column, id_column):
    unique_values = df[name_column].unique()
    id_mapping = {name: idx + 1 for idx, name in enumerate(unique_values)}
    df[id_column] = df[name_column].map(id_mapping).astype(int)
    return df, id_mapping

# Add IDs to each table
writers_df, writer_mapping = add_unique_id(writers_df, 'writer_name', 'writer_id')
genres_df, genre_mapping = add_unique_id(genres_df, 'genre', 'genre_id')
production_companies_df, company_mapping = add_unique_id(production_companies_df, 'company_name', 'company_id')
languages_df, language_mapping = add_unique_id(languages_df, 'languages', 'language_id')
country_df, country_mapping = add_unique_id(country_df, 'country', 'country_id')
director_df, director_mapping = add_unique_id(director_df, 'director_name', 'director_id')


In [112]:
master_df = az_movies[['title_id', 'Title', 'release_date', 'run_time']]

# Merge IDs into master_df
master_df = master_df.merge(writers_df[['title_id', 'writer_id']], on='title_id', how='left')
master_df = master_df.merge(genres_df[['title_id', 'genre_id']], on='title_id', how='left')
master_df = master_df.merge(production_companies_df[['title_id', 'company_id']], on='title_id', how='left')
master_df = master_df.merge(languages_df[['title_id', 'language_id']], on='title_id', how='left')
master_df = master_df.merge(country_df[['title_id', 'country_id']], on='title_id', how='left')
master_df = master_df.merge(director_df[['title_id', 'director_id']], on='title_id', how='left')

# Ensure all ID columns are integers (replace NaN with 0 before converting)
id_columns = ['writer_id', 'genre_id', 'company_id', 'language_id', 'country_id', 'director_id']
master_df[id_columns] = master_df[id_columns].fillna(0).astype(int)


In [113]:
languages_df

,title_id,languages,language_id
0,0,English,1
1,1,Japanese,2
2,2,English,1
3,3,Japanese,2
4,4,English,1
...,...,...,...
965,995,English,1
966,996,English,1
967,997,English,1
968,998,Portuguese,26


In [114]:
production_companies_df

,title_id,company_name,company_id
0,1,Netflix,1
1,2,Warner Bros. Pictures,2
2,3,Aniplex,3
3,4,Universal Pictures,4
4,5,Seven Bucks Productions,5
...,...,...,...
958,995,Forgan-Smith Entertainment,458
959,996,Fifth Season,459
960,997,Tandem Communications,460
961,998,Globo Movies,461


In [115]:
country_df

,title_id,country,country_id
0,1,Japan,1
1,3,Japan,1
2,4,United States of America,2
3,5,United States of America,2
4,8,France,3
...,...,...,...
497,989,United States of America,2
498,990,United States of America,2
499,991,United States of America,2
500,994,United States of America,2


In [116]:
director_df

,title_id,director_name,director_id
0,1,Seiji Tanaka,1
1,2,Bong Joon Ho,2
2,3,Shunsuke Nakashige,3
3,4,Jon M. Chu,4
4,5,Jake Kasdan,5
...,...,...,...
904,995,Gábor Csupó,647
905,996,Emily Ting,648
906,997,Darrell James Roodt,649
907,998,Cláudio Torres,650


<h5 style="color:#A64D79;font-style:italic;font-weight:bold;">Using Column Splitting to Create a Separate Ratings DataFrame</h5>

In [117]:
rating_columns = ['title_id', 'user_rating', 'imdb_rating', 'Metascore', 'rt_score']
rating_df = az_movies[rating_columns].copy()
az_movies = az_movies.drop(columns=['user_rating', 'imdb_rating', 'Metascore', 'rt_score'])

In [118]:
rating_df

,title_id,user_rating,imdb_rating,Metascore,rt_score
0,0,8.0,4.9,0.0,0.0
1,1,6.8,0.0,0.0,0.0
2,2,7.1,7.3,75.0,89.0
3,3,6.8,8.6,0.0,0.0
4,4,6.9,8.0,73.0,88.0
...,...,...,...,...,...
995,995,6.5,6.0,23.0,0.0
996,996,6.8,6.3,92.0,0.0
997,997,5.3,3.6,0.0,0.0
998,998,7.2,7.0,0.0,0.0


In [119]:
az_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 970 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title_id      970 non-null    int64 
 1   Title         970 non-null    object
 2   release_date  970 non-null    object
 3   run_time      970 non-null    object
dtypes: int64(1), object(3)
memory usage: 37.9+ KB


<h4 style="color:#A64D79;font-style:italic; font-weight:bold;">Using SQLite to Store and Query Normalized Movie Data</h4>

In [120]:
import sqlite3

In [121]:
DATA_FOLDER = os.path.join('../data/clean')

In [122]:
conn = sqlite3.connect(os.path.join(DATA_FOLDER, './Movies.db'))

In [123]:
master_df.to_sql('movies_master', conn, if_exists='replace', index=False)

6442

In [124]:
pd.read_sql('SELECT * FROM movies_master LIMIT 5', conn)

,title_id,Title,release_date,run_time,writer_id,genre_id,company_id,language_id,country_id,director_id
0,0,Flow,1996-01-01,1h 20min,0,1,0,1,0,0
1,0,Flow,1996-01-01,1h 20min,0,2,0,1,0,0
2,0,Flow,1996-01-01,1h 20min,0,3,0,1,0,0
3,0,Flow,1996-01-01,1h 20min,0,4,0,1,0,0
4,1,Demon City,2025-02-26,1h 46min,0,5,1,2,1,1


In [125]:
genres_df.to_sql('geners', conn, if_exists='replace', index=False)

3506

In [126]:
pd.read_sql('SELECT * FROM geners LIMIT 5', conn)

,title_id,genre,genre_id
0,0,Drama,1
1,0,Comedy,2
2,0,Horror,3
3,0,Crime,4
4,1,Action,5


In [127]:
languages_df.to_sql('languages', conn, if_exists='replace', index=False)

970

In [128]:
pd.read_sql('SELECT * FROM languages LIMIT 5', conn)

,title_id,languages,language_id
0,0,English,1
1,1,Japanese,2
2,2,English,1
3,3,Japanese,2
4,4,English,1


In [129]:
rating_df.to_sql('ratings',conn, if_exists='replace', index=False)

970

In [130]:
pd.read_sql('SELECT * FROM ratings LIMIT 5', conn)

,title_id,user_rating,imdb_rating,Metascore,rt_score
0,0,8.0,4.9,0.0,0.0
1,1,6.8,0.0,0.0,0.0
2,2,7.1,7.3,75.0,89.0
3,3,6.8,8.6,0.0,0.0
4,4,6.9,8.0,73.0,88.0


In [131]:
production_companies_df.to_sql('pro_company',conn, if_exists='replace', index=False)

963

In [132]:
pd.read_sql('SELECT * FROM pro_company LIMIT 5', conn)

,title_id,company_name,company_id
0,1,Netflix,1
1,2,Warner Bros. Pictures,2
2,3,Aniplex,3
3,4,Universal Pictures,4
4,5,Seven Bucks Productions,5


In [133]:
country_df.to_sql('pro_country',conn, if_exists='replace', index=False)

502

In [134]:
pd.read_sql('SELECT * FROM pro_country LIMIT 5', conn)

,title_id,country,country_id
0,1,Japan,1
1,3,Japan,1
2,4,United States of America,2
3,5,United States of America,2
4,8,France,3


In [135]:
writers_df.to_sql('screen_writers',conn, if_exists='replace', index=False)

1198

In [136]:
pd.read_sql('SELECT * FROM screen_writers LIMIT 5', conn)

,title_id,writer_name,writer_id
0,4,Winnie Holzman,1
1,4,Dana Fox,2
2,5,Chris Morgan,3
3,6,Phoebe Gittins,4
4,6,Arty Papageorgiou,5


In [137]:
director_df.to_sql('directors',conn, if_exists='replace', index=False)

909

In [138]:
pd.read_sql('SELECT * FROM directors LIMIT 5', conn)

,title_id,director_name,director_id
0,1,Seiji Tanaka,1
1,2,Bong Joon Ho,2
2,3,Shunsuke Nakashige,3
3,4,Jon M. Chu,4
4,5,Jake Kasdan,5


In [139]:
conn.commit()

In [140]:
conn.close()